In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import json
import torch
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from nerfstudio.utils.eval_utils import eval_setup
from plane_nerf.inerf_trainer import load_data_into_trainer
from plane_nerf.inerf_utils import inerf
from plane_nerf.inerf_utils import load_eval_image_into_pipeline

/home/user/.local/lib/python3.10/site-packages/tinycudann/modules.py:31: UserWarning: System has multiple GPUs with different compute capabilities: [61, 52]. Using compute capability 52 for best compatibility. This may result in suboptimal performance.
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
os.chdir('/workspace/plane-nerf')
MODEL_PATH = "/workspace/plane-nerf/outputs/jackal_floor_training_data_1/plane-nerf-fast/2024-03-13_105236"
DATA_PATH = "/stored_data/jackal_one_frame_color"
GROUND_TRUTH_PATH = os.path.join(DATA_PATH, "ground_truth.json")
config_path = os.path.join(MODEL_PATH, "config.yml")
config, pipeline, _, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )
transform_file_path = "transforms.json"
with open(os.path.join(DATA_PATH, transform_file_path)) as f:
    transform = json.load(f)

pipeline = load_eval_image_into_pipeline(pipeline,DATA_PATH,transform)

config.pipeline.datamanager.pixel_sampler.num_rays_per_batch = 4096 

trainer = load_data_into_trainer(
    config,
    pipeline,
    plane_optimizer = True
)
trainer.pipeline.datamanager.KERNEL_SIZE = 5
trainer.pipeline.datamanager.THRESHOLD = 40
trainer.pipeline.datamanager.METHOD = "sift"

tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Fal

Loading latest checkpoint from load_dir

──────────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────────

No checkpoint directory found at                                            
               outputs/jackal_floor_training_data_1/plane-nerf-fast/2024-03-13_105236/nerfstudio_models,

Please make sure the checkpoint exists, they should be generated periodically during training

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/user/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553 in run_code │
│                                                                                                  │
│   3550 │   │   │   │   if async_:                                                                │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 DATA_PATH = "/stored_data/jackal_one_frame_color"                                           │
│    4 GROUND_TRUTH_PATH = os.path.join(DATA_PATH, "ground_truth.json")                            │
│    5 config_path = os.path.join(MODEL_PATH, "config.yml")                                        │
│ ❱  6 config, pipeline, _, _ = eval_setup(                                                        │
│    7 │   │   │   │   │   │   Path(config_path),                                                  │
│    8 │   │   │   │   │   │   test_mode="inference",                                              │
│    9 │   │   │   │   │   )                                                                       │
│                                                                                                  │
│ /home/user/nerfstudio/nerfstudio/utils/eval_utils.py:110 in eval_setup                           │
│                                                                                                  │
│   107 │   pipeline.eval()                                                                        │
│   108 │                                                                                          │
│   109 │   # load checkpointed information                                                        │
│ ❱ 110 │   checkpoint_path, step = eval_load_checkpoint(config, pipeline)                         │
│   111 │                                                                                          │
│   112 │   return config, pipeline, checkpoint_path, step                                         │
│   113                                                                                            │
│                                                                                                  │
│ /home/user/nerfstudio/nerfstudio/utils/eval_utils.py:55 in eval_load_checkpoint                  │
│                                                                                                  │
│    52 │   │   │   │   "Please make sure the checkpoint exists, they should be generated period   │
│    53 │   │   │   │   justify="center",                                                          │
│    54 │   │   │   )                                                                              │
│ ❱  55 │   │   │   sys.exit(1)                                                                    │
│    56 │   │   load_step = sorted(int(x[x.find("-") + 1 : x.find(".")]) for x in os.listdir(con   │
│    57 │   else:                                                                                  │
│    58 │   │   load_step = config.load_step                 

/home/user/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
with open(GROUND_TRUTH_PATH) as f:
    ground_truth_transform = json.load(f)
ground_truth_poses = []
tf = ground_truth_transform["frames"][0]["transform_matrix"]
tf = np.asarray(tf)
tf = tf[:3, :4 ]
ground_truth_poses.append(tf)
ground_truth_poses = torch.tensor(ground_truth_poses).to(trainer.pipeline.device)

/tmp/ipykernel_1512872/1219336787.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  ground_truth_poses = torch.tensor(ground_truth_poses).to(trainer.pipeline.device)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│   5 tf = np.asarray(tf)                                                                          │
│   6 tf = tf[:3, :4 ]                                                                             │
│   7 ground_truth_poses.append(tf)                                                                │
│ ❱ 8 ground_truth_poses = torch.tensor(ground_truth_poses).to(trainer.pipeline.device)            │
│   9                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'trainer' is not defined

In [5]:
import time
start = time.time()
ans = inerf(trainer,GROUND_TRUTH_POSE=ground_truth_poses)
end = time.time()
print("Time taken for inference: ", end-start)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 import time                                                                                  │
│   2 start = time.time()                                                                          │
│ ❱ 3 ans = inerf(trainer,GROUND_TRUTH_POSE=ground_truth_poses)                                    │
│   4 end = time.time()                                                                            │
│   5 print("Time taken for inference: ", end-start)                                               │
│   6                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'trainer' is not defined

In [6]:
#Plot store_t_diff and store_r_diff with  respect to training iteration in 2 subplots

plotting_data = np.asarray(ans["store"].to("cpu"))
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(plotting_data[:, 0], plotting_data[:, 1])
plt.xlabel("Training iteration")
plt.ylabel("Absolute translation error")
plt.subplot(1, 2, 2)
plt.plot(plotting_data[:, 0], plotting_data[:, 2])
plt.xlabel("Training iteration")
plt.ylabel("Absolute rotation error")
plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 #Plot store_t_diff and store_r_diff with  respect to training iteration in 2 subplots       │
│    2                                                                                             │
│ ❱  3 plotting_data = np.asarray(ans["store"].to("cpu"))                                          │
│    4 plt.figure(figsize=(15, 5))                                                                 │
│    5 plt.subplot(1, 2, 1)                                                                        │
│    6 plt.plot(plotting_data[:, 0], plotting_data[:, 1])                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ans' is not defined

In [7]:
print(ans["store"])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(ans["store"])                                                                          │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ans' is not defined